In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import average_precision_score, precision_score, recall_score
from HybridRetrieval import HybridRetrieval  # تأكد أن هذا الملف موجود في نفس المجلد
from Data_Processing_Antique import data_processing_antique

# تحميل البيانات
print("📂 تحميل البيانات...")
docs_df = pd.read_csv("cleaned_quora.csv").dropna(subset=["cleaned_text"]).reset_index(drop=True)
queries_df = pd.read_csv("queries.tsv", sep="\t").dropna(subset=["text"])
qrels_df = pd.read_csv("qrels.tsv", sep="\t")
print("✅ تم تحميل الوثائق والاستعلامات والعلاقات.")

# بناء قاموس العلاقات
print(" بناء قاموس العلاقات (qrels)...")
qrels_dict = {}
for _, row in qrels_df.iterrows():
    qrels_dict.setdefault(row["query_id"], set()).add(row["doc_id"])
print("✅ تم بناء قاموس qrels.")

# تحميل النموذج الهجين
print(" تحميل نماذج TF‑IDF وWord2Vec وتمثيلات الوثائق...")

hybrid_model = HybridRetrieval(
    dataset='quora',
    tfidf_index_path='tfidf_quora.npz',
    tfidf_vectorizer_path='vectorizer_quora.joblib',
    w2v_model_path='word2vec_quora.model',
    doc_vectors_path='quora_doc_vectors.joblib'
)


# بدء التقييم
print("بدء التقييم باستخدام التمثيل التسلسلي (Serial Fusion)...")

map_scores, mrr_scores, precision_scores, recall_scores = [], [], [], []

for i, query_row in queries_df.iterrows():
    qid = query_row["query_id"]
    raw_query = query_row["text"]

    # استرجاع أفضل الوثائق
    top_docs = hybrid_model.retrieve(raw_query, alpha=0.7, prefilter_k=100, rerank_k=10)
    doc_id_lookup = docs_df["doc_id"].tolist()
    top_doc_ids = [doc_id_lookup[idx] for idx, _ in top_docs]

    relevant_docs = qrels_dict.get(qid, set())
    if not relevant_docs:
        continue

    y_true = [1 if doc_id in relevant_docs else 0 for doc_id in top_doc_ids]
    y_scores = [score for _, score in top_docs]

    if sum(y_true) == 0:
        continue

    map_scores.append(average_precision_score(y_true, y_scores))
    mrr_scores.append(1 / (np.where(np.array(y_true) == 1)[0][0] + 1))
    precision_scores.append(precision_score(y_true, [1] * len(y_true)))
    recall_scores.append(recall_score(y_true, [1] * len(y_true)))

  

# حساب المتوسطات
map_score = np.mean(map_scores)
mrr_score = np.mean(mrr_scores)
precision_score_10 = np.mean(precision_scores)
recall_score_10 = np.mean(recall_scores)

# طباعة النتائج
print("\n📊 النتائج النهائية للتقييم:")
print(f"🔹 MAP  : {map_score:.4f}")
print(f"🔹 MRR  : {mrr_score:.4f}")
print(f"🔹 P@10 : {precision_score_10:.4f}")
print(f"🔹 R@10 : {recall_score_10:.4f}")

# حفظ النتائج
results = {
    "MAP": round(map_score, 4),
    "MRR": round(mrr_score, 4),
    "P@10": round(precision_score_10, 4),
    "R@10": round(recall_score_10, 4)
}

with open("evaluation_results_Hybrid_Antique.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4)

print("\n✅ تم حفظ النتائج في evaluation_results_Hybrid_Quora.json")


📂 تحميل البيانات...
✅ تم تحميل الوثائق والاستعلامات والعلاقات.
 بناء قاموس العلاقات (qrels)...
✅ تم بناء قاموس qrels.
 تحميل نماذج TF‑IDF وWord2Vec وتمثيلات الوثائق...
بدء التقييم باستخدام التمثيل التسلسلي (Serial Fusion)...

📊 النتائج النهائية للتقييم:
🔹 MAP  : 0.7323
🔹 MRR  : 0.7749
🔹 P@10 : 0.1225
🔹 R@10 : 1.0000

✅ تم حفظ النتائج في evaluation_results_Hybrid_Quora.json
